# Introduction to ipyautoui

ipyautoui is a Python package that provides a simple way to create interactive user interfaces in Jupyter notebooks. It is built on top of Jupyter widgets and Pydantic and provides a higher-level API for creating interactive widgets.  It is particularly well suited for creating settings panels.


## TO DO IN THIS NOTEBOOK
- [ ] Add a brief description of the package and its history?
- [ ] Show how to use ipyautoui to create a simple settings panel relatively quickly
  - [ ] Mention the active development of the package and the fact that it is still a bit buggy
  - [ ] Mention other features of Pydantic that may be relevant to our dashboard
- [ ] Have them replace existing settings widgets with control widget generated with ipyautoui
- [ ] Connect new control widget to dashboard

# APPARENTLY WE CANT EXPORT FROM TWO NOTEBOOKS

In [ ]:
#| default_exp widgets_pydantic

## NEED MORE INTRO BEFORE WE DO THIS

In [ ]:
from ipyautoui import AutoUi
from dashboard.widgets_pydantic import DataSelectorModelDraft6

In [ ]:
controls = AutoUi(DataSelectorModelDraft6)

In [ ]:
controls

In [ ]:
from pydantic import BaseModel, Field, model_validator
from typing import Annotated

In [ ]:
#| export

class DataSelectorModel(BaseModel, validate_assignment=True):
    year_range: Annotated[
        tuple[int, int],
        Field(json_schema_extra=dict(autoui='ipywidgets.IntRangeSlider'))
    ] = (1800, 2000)
    window_size: Annotated[int, Field(ge=2, le=100, json_schema_extra=dict(autoui='ipywidgets.BoundedIntText'))] = 2
    polynomial_order: Annotated[int, Field(ge=1, le=10)] = 1

    # mode="after" means the validator runs after pydantic has checked that the individual
    # fields have values that are valid.
    @model_validator(mode="after")
    def limit_polynomial_order(self):
        
        if self.polynomial_order > self.window_size - 1:
            # Handle a bad polynomial order or window size
            raise ValueError("Polynomial order must be smaller than window size")
            
        # If we got this far the polynomial order is consistent with the window size
        # so return self. Failing to return self will end up causing an error.
        return self

In [ ]:
DataSelectorModel.model_json_schema()

In [ ]:
ui = AutoUi(DataSelectorModel)
ui

In [ ]:
ui.value

In [ ]:
ui.di_widgets

In [ ]:
from ipyautoui import demo
demo()

In [ ]:
from nbdev.export import nb_export

nb_export('03b_ipyautoui.ipynb', 'dashboard')